In [1]:
!ls ./static

Pandora_dup_img.png Picture1.png


<img src=./static/Picture1.png>

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [3]:
url='https://www.pandora.com/playlist/PL:1688849921155861:905692073'

In [8]:
def dupli_check(url):
    #equest playlist information by using ‘requests’ and parse with 'BeautifulSoup'
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html")
    page_str=str(soup)
    json_dict=page_str.split('var ')[4].replace(';\n    ','').replace('storeData = ','')
    
    #Convert this information to Dictionary using Json
    dic=json.loads(json_dict)
    
    #This dictionary contains two keys:
    #['v4/catalog/annotateObjects', 'v7/playlists/getTracks']
    #Our plans is to create two DataFrame, for each key in the dictionary, and merging it at the end:
    df_tracks=pd.DataFrame(dic['v7/playlists/getTracks'][0]['tracks'])

    #Each song is displayed as a 'trackPandoraId', so we need to pull song information from the other part of dictionary, annotateObjects.
    df_info=pd.DataFrame.from_dict(dic['v4/catalog/annotateObjects'][0], orient='index')
    df_info=df_info.reset_index()
    df_info.rename(columns={'index':'trackPandoraId'}, inplace=True)
    
    #Now we have all the information we needed. Let’s merge these DataFrames:
    df=df_tracks.merge(df_info, left_on='trackPandoraId', right_on='trackPandoraId').sort_values(by=['itemId'])

    #Simply, use groupby function to display how many duplicates are in this playlist:
    df_numbers=df[['name','artistName','itemId']].groupby(['name','artistName']).count().sort_values(by='itemId', ascending=False)

    #Use duplicated function to see where the duplicate songs are located in the playlist:
    df['duplicated']=df.duplicated(subset='name')
    df_loc=df[['name','duplicated']]
    
    return df_numbers, df_loc

In [9]:
df_numbers, df_loc = dupli_check(url)

In [10]:
df_numbers

,,itemId
name,artistName,
Faithfully,Journey,5
Fly Me To The Moon,Frank Sinatra,3
Man In The Mirror,Michael Jackson,3
Misty,Ella Fitzgerald,3
Candle In The Wind (Remastered),Elton John,1
Don't Let The Sun Go Down On Me,Elton John,1
Lights,Journey,1
Piano Man,Billy Joel,1
She's Always A Woman (Live),Billy Joel,1


In [11]:
df_loc

,name,duplicated
0,Piano Man,False
1,Candle In The Wind (Remastered),False
2,Lights,False
3,With A Little Help From My Friends (Remix),False
4,She's Always A Woman (Live),False
5,Don't Let The Sun Go Down On Me,False
6,Misty,False
7,Misty,True
8,Misty,True
9,Fly Me To The Moon,False
